### Hyperparameters to reproduce results of paper "Computing the untruncated signature kernel..."

In [2]:
# name, sigma, ll, at = 'ArticularyWordRecognition', 6e-1, False, False
# name, scale, rbf, sigma, ll, at = 'BasicMotions', 1e-1, True, 7e1, False, False
# name, scale, rbf, sigma, ll, at = 'Cricket', 1e-1, True, 1e0, False, False # subsample_len every 10 time steps
# name, scale, rbf, sigma, ll, at = 'Epilepsy', 1e-1, True, 1e0, False, False
# name, scale, rbf, sigma, ll, at = 'ERing', 1e-2, True, 9e-2, False, True
# name, scale, rbf, sigma, ll, at = 'Heartbeat', 1e-1, True, 1e1, False, True
# name, scale, rbf, sigma, ll, at = 'Libras', 1e-1, True, 3e-2, False, False
# name, scale, rbf, sigma, ll, at = 'NATOPS', 1e-1, True, 2.25e-1, False, True
# name, scale, rbf, sigma, ll, at = 'RacketSports', 1e-1, True, 1e1, False, False
# name, scale, rbf, sigma, ll, at = 'SelfRegulationSCP1', 1e-1, True, 2.5e2, False, False # subsample_len every 2 time steps
# name, scale, rbf, sigma, ll, at = 'UWaveGestureLibrary', 1e0, True, 3.5e1, False, True # subsample_len every 5 time steps
# name, scale, rbf, sigma, ll, at = 'ElectricDevices', 1e0, False, 3.5e1, False, True

In [3]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import iisignature
import torch 

from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, classification_report
from tslearn.datasets import UCR_UEA_datasets
from tslearn.preprocessing import TimeSeriesScalerMinMax, TimeSeriesScalerMeanVariance
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit

from tslearn.svm import TimeSeriesSVC

import sigkernel

In [19]:
X_train, Y_train, X_test, Y_test = UCR_UEA_datasets(use_cache=True).load_dataset('Epilepsy')

In [20]:
X_train.shape

(137, 206, 3)

In [22]:
# store classification results
final = {}

datasets = tqdm([
#             'ArticularyWordRecognition'
#             'BasicMotions',
#             'Cricket',
            'Epilepsy'
           ])

for name in datasets:
    datasets.set_description(f"processing dataset: {name}")
    
    # load data
    X_train, Y_train, X_test, Y_test = UCR_UEA_datasets(use_cache=True).load_dataset(name)
    
    # subsample every time steps if length exceeds 1000
    if X_train.shape[1]>1000:
        subsample = 10
        X_train = X_train[:,::subsample,:]
        X_test = X_test[:,::subsample,:]

    # standard-scale inputs
    x_train = TimeSeriesScalerMeanVariance().fit_transform(X_train)
    x_test = TimeSeriesScalerMeanVariance().fit_transform(X_test)
    
    # path-transforms
    x_train = sigkernel.transform(x_train, at=True, scale=.1)
    x_test = sigkernel.transform(x_test, at=True, scale=.1)
    
    # encode outputs as labels
    y_train = LabelEncoder().fit_transform(Y_train)
    y_test = LabelEncoder().fit_transform(Y_test)

    # define grid-search hyperparameters for SVC (common to all kernels)
    svc_parameters = {'C': np.logspace(0, 4, 5), 'gamma': np.logspace(-3, 3, 7)}

    #==================================================================================
    # Linear, RBF and GAK kernels
    #==================================================================================
#             for ker in tqdm(['linear', 'rbf', 'gak'], desc=f'4th loop (standard kernels)', leave=False):

#                 # SVC tslearn estimator
#                 svc = TimeSeriesSVC(kernel=ker, gamma='auto', decision_function_shape='ovo')
#                 svc_model = GridSearchCV(estimator=svc, param_grid=svc_parameters, cv=TimeSeriesSplit(n_splits=5), n_jobs=-1)
#                 svc_model.fit(x_train, y_train)

#                 # store results
#                 final[(ker, f'add-time: {at}', f'lead-lag: {ll}')] = svc_model.score(x_test, y_test)

    #==================================================================================
    # Truncated signature kernels
    #==================================================================================
#             # set max signature truncation
#             max_depth = 7 if x_train.shape[-1] < 10 else 6

#             for d in tqdm(range(2,max_depth), desc='4th loop (truncated sig kernels)', leave=False):

#                 # truncated signatures
#                 sig_train = iisignature.sig(x_train, d)
#                 sig_test = iisignature.sig(x_test, d)

#                 # SVC tslearn estimator
#                 svc = TimeSeriesSVC(kernel='linear', decision_function_shape='ovo')
#                 svc_model = GridSearchCV(estimator=svc, param_grid=svc_parameters, cv=5, n_jobs=-1)
#                 svc_model.fit(sig_train, y_train)

#                 # store results
#                 final[(f'trunc-sig-ker-{d}', f'add-time: {at}', f'lead-lag: {ll}')] = svc_model.score(sig_test, y_test)

    #==================================================================================
    # Signature PDE kernel
    #==================================================================================
    # define grid for sigmas
    sigmas = tqdm(list(np.linspace(0,1,41)[1:]) + [1.] + list(np.linspace(0,200,41)[1:]), leave=False)
    best_score, best_sigma = 0., 0.

    for sigma in sigmas:
        sigmas.set_description(f"sig PDE kernel sigma: {sigma}")

        # define static kernel
        static_kernel = sigkernel.RBFKernel(sigma=sigma)

        # initialize corresponding signature PDE kernel
        signature_kernel = sigkernel.SigKernel(static_kernel, dyadic_order=0)
        
        if x_train.shape[0]<110 and x_train.shape[1]<1000:
            # move to torch
            x_train = torch.tensor(x_train, dtype=torch.float64, device=torch.device('cuda' if torch.cuda.is_available() else 'cpu'))
            x_test = torch.tensor(x_test, dtype=torch.float64, device=torch.device('cuda' if torch.cuda.is_available() else 'cpu'))
        else:
            # move to torch
            x_train = torch.tensor(x_train, dtype=torch.float64, device='cpu')
            x_test = torch.tensor(x_test, dtype=torch.float64, device='cpu')
                    
        # compute Gram matrix on train data
        G_train = signature_kernel.compute_Gram(x_train, x_train, sym=True).cpu().numpy()

        # compute Gram matrix on test data
        G_test = signature_kernel.compute_Gram(x_test, x_train, sym=False).cpu().numpy()

        # SVC sklearn estimator
        svc = TimeSeriesSVC(kernel='precomputed', decision_function_shape='ovo')
        svc_model = GridSearchCV(estimator=svc, param_grid=svc_parameters, cv=5, n_jobs=-1)
        svc_model.fit(G_train, y_train)

        # choose best model
        _score = svc_model.score(G_test, y_test)
        if _score > best_score:
            best_score = _score
            best_sigma = sigma
            
        del G_train
        del G_test
        torch.cuda.empty_cache()

    # store results
    final[('sig-ker-PDE', f'best sigma: {best_sigma}')] = best_score
            
    final_csv = pd.DataFrame.from_dict(final, orient='index').rename(columns={0:'accuracy (%)'})
    final_csv.to_csv(f'../results/_{name}.csv')

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/81 [00:00<?, ?it/s]

RuntimeError: CUDA out of memory. Tried to allocate 5.94 GiB (GPU 0; 15.90 GiB total capacity; 9.65 GiB already allocated; 5.50 GiB free; 9.70 GiB reserved in total by PyTorch)

In [9]:
final

{('sig-ker-PDE', 'best sigma: 0.47500000000000003'): 0.9722222222222222}